## Efficient Pandas: Using Chunksize for Large Data Sets   
We use the Movielens data set a large data set, used for building Recommender Systems, which contains over 20 million movie ratings by over 138,000 users, covering over 27,000 different movies. So let’s extract it using wget.

In [2]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip

--2020-07-23 14:01:18--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: 'moviedataset.zip'

moviedataset.zip    100%[===================>] 152.88M   620KB/s    in 31s     

2020-07-23 14:01:49 (4.98 MB/s) - 'moviedataset.zip' saved [160301210/160301210]



In [3]:
print('unziping ...')
!unzip -o -j moviedataset.zip

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
ratings_df = pd.read_csv('ratings.csv')
print(ratings_df.shape)

(22884377, 4)


This is a lot of data for our computer’s memory to handle. We can using method head() to view first 5 lines

In [7]:
pd.read_csv('ratings.csv').head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


1. What’s the most common movie rating from 0.5 to 5.0
2. What’s the average movie rating for most movies.   

Let’s check the memory consumption of the ratings_df data frame


In [9]:
ratings_memory = ratings_df.memory_usage().sum()
# Let's print out the memory consumption
print('Total Current memory is-', ratings_memory,'Bytes.')
# Finally, let's see the memory usage of each dimension.
ratings_df.memory_usage()

Total Current memory is- 732300192 Bytes.


Index              128
userId       183075016
movieId      183075016
rating       183075016
timestamp    183075016
dtype: int64

Let’s get the different rating keys on the scale, from 0.5 to 5.0

In [10]:
# Let's get a list of the rating scale or keys
rate_keys = list(ratings_df['rating'].unique())
# let's sort the ratings keys from highest to lowest.
rate_keys = sorted(rate_keys, reverse=True)  
print(rate_keys)

[5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5]


1. What’s the most common movie rating from 0.5 to 5.0


In [17]:
ratings_dict = {}
for i in rate_keys: ratings_dict[i] = 0


# Initialise a variable to compute average bytes per chunk
ave_bytes = 0
# then we initialise our loop counter
count = 0
# This enumerate function selects repeated chunks of 1,000,000 rows of data
for index, chunk in enumerate(pd.read_csv('ratings.csv', chunksize= 1000000),start=1):
# We add total memory per chunk to ave_bytes
    ave_bytes += chunk.memory_usage().sum()
    for i in rate_keys:
        count = len(chunk[chunk['rating'] == i])
        ratings_dict[i] += count
    
print("Total number of chunks:",index)
ave_bytes = ave_bytes / index
print("Average bytes per loop:",ave_bytes)
print(ratings_dict)

Total number of chunks: 23
Average bytes per loop: 31839265.04347826
{5.0: 3358218, 4.5: 1813922, 4.0: 6265623, 3.5: 2592375, 3.0: 4783899, 2.5: 1044176, 2.0: 1603254, 1.5: 337605, 1.0: 769654, 0.5: 315651}


Let look at the last chunk

In [18]:
chunk

,userId,movieId,rating,timestamp
22000000,237997,1610,4.0,945894162
22000001,237997,1617,5.0,945891177
22000002,237997,1672,4.0,945894070
22000003,237997,1673,5.0,945893844
22000004,237997,1682,4.0,945894098
...,...,...,...,...
22884372,247753,49530,5.0,1430437962
22884373,247753,69481,3.0,1430437984
22884374,247753,74458,4.0,1430437968
22884375,247753,76093,5.0,1430437811


In [14]:
for i in rate_keys:
    count = len(chunk[chunk['rating'] == i])
    ratings_dict[i] += count
    
print("Total number of chunks:",index)
ave_bytes = ave_bytes / index
print("Average bytes per loop:",ave_bytes)
print(ratings_dict)

Total number of chunks: 23
Average bytes per loop: 31839265.04347826
{5.0: 131091, 4.5: 70995, 4.0: 243091, 3.5: 99331, 3.0: 186158, 2.5: 39769, 2.0: 60563, 1.5: 12306, 1.0: 28753, 0.5: 12320}


Note that By specifying chunksize in read_csv, the return value will be an iterable object of type TextFileReader .Specifying iterator=True will also return the TextFileReader object:   

### Example of passing chunksize to read_csv
reader = pd.read_csv(’some_data.csv’, chunksize=100)
**Above code reads first 100 rows, if you run it in a loop, it reads the next 100** and so on
### Example of iterator=True. Note iterator=False by default.   
reader = pd.read_csv('some_data.csv', iterator=True)   
reader.get_chunk(100)  
**This gets the first 100 rows, running through a loop gets the next 100 rows** and so on.  

 Both chunksize=100 and reader.get_chunk(100) return same TextFileReader object.   
 
Let’s confirm we got the complete ratings from the exercise we did above. Total ratings should be equal to the number of rows in the ratings_df.

In [19]:
sum(list(ratings_dict.values())) == len(ratings_df)

True

In [20]:
ratings_dict

{5.0: 3358218,
 4.5: 1813922,
 4.0: 6265623,
 3.5: 2592375,
 3.0: 4783899,
 2.5: 1044176,
 2.0: 1603254,
 1.5: 337605,
 1.0: 769654,
 0.5: 315651}

In [21]:
# We use the operator module to easily get the max and min values
import operator
max(ratings_dict.items(), key=operator.itemgetter(1))


(4.0, 6265623)

Alternative

In [25]:
max(ratings_dict, key=ratings_dict.get),ratings_dict[max(ratings_dict, key=ratings_dict.get)]

(4.0, 6265623)

**2. What’s the average movie rating for most movies**     
This is the Weighted-Average of the distribution.
This simply means we multiply each rating key by the number of times it was rated and we add them all together and divide by the total number of ratings.

In [33]:
ratings_dict_df = pd.DataFrame(list(ratings_dict.items()) ,columns = ['Rating_Keys','Count'])

# First we find the sum of the product of rate keys and corresponding values.
product = sum((ratings_dict_df.Rating_Keys * ratings_dict_df.Count))
# Let's divide product by total ratings.
weighted_average = product / len(ratings_df)
# Then we display the weighted-average below.
weighted_average

3.5260770044122243

**What percentage of movie ratings does each rating key hold?**

In [34]:
ratings_dict_df['Percent'] = ratings_dict_df['Count'].apply(lambda x: (x / (len(ratings_df)) * 100))
ratings_dict_df

,Rating_Keys,Count,Percent
0,5.0,3358218,14.674719
1,4.5,1813922,7.926464
2,4.0,6265623,27.379478
3,3.5,2592375,11.328143
4,3.0,4783899,20.904650
5,2.5,1044176,4.562833
6,2.0,1603254,7.005889
7,1.5,337605,1.475264
8,1.0,769654,3.363229
9,0.5,315651,1.379330


**To find the percentage of movies that are rated at least average (3.5)**, we simply sum the percentages of movie keys 3.5 to 5.0.

In [35]:
sum(ratings_dict_df[ratings_dict_df.Rating_Keys >= 3.5]['Percent'])

61.308804692389046

From these exercises, we can infer that on a scale of 5.0, most movies are rated 4.0 and the average rating for movies is 3.5 and finally, over 61.3% of all movies produced have a rating of at least 3.5.